In [1]:
# *Väestöennusteisiin tarvittavien Tilastokeskuskoodien ajo ja haku;
# Haetaan Tilastokeskukselta kuntien välinen muutto - tarvitaan koodissa 3_SEUDUNENN

# 2025 Juuso Heinisuo - Kangasalan kaupunki
# https://github.com/kangasalakehitys/vaestoennuste

from pyjstat import pyjstat
import requests
import re
from functools import reduce
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Kirjautumistunnukset
un = 'XXXXX'
pw = 'XXXX'

#Exceliin ennustevuodet
file_exc = 'Työmarkkina_tulomuutto_Ve1.xlsx'
ENNUSTEVUODET = ["2025","2026","2027","2028","2029","2030","2031","2032","2033","2034","2035"]
ALUE = ["1","1","1","1","1","1","1","1","1","1","1"]
TYYPPI = ["1","1","1","1","1","1","1","1","1","1","1"]

# Tilastokeskuksen kyselyyn parametrit
OMA_KUNTA = ["KU211"]
IKÄRYHMÄT = ["SSS","0-4","5-9","10-14","15-19","20-24","25-29","30-34","35-39","40-44","45-49","50-54","55-59","60-64","65-69","70-74","75-"]
VUODET = ["2020","2021","2022","2023","2024"]

# Valitaan oma maakunta
OMA_MAAKUNTA = '06'

# Rajapinnan osoite 12pn -- Kuntien välisen muuttoliikkeen tulomuutto tulokunnan, muuttaneiden iän ja kielen mukaan lähtökunnittain, 2010-2024
POST_URL = 'https://pxhopea2.stat.fi/PXWeb/api/v1/fi/Muuttoliike_ja_vaestonmuutokset/Muuttoliike_ja_vaestonmuutokset_aikasarja/Muuttoliike/muvamuu_12pn.px'
headers = {'Content-Type': 'application/json', 'un': un, 'pw': pw }

#Haetaan kunta-maakunta-avain
KUNTA_MAAKUNTA_CSV = 'kunta_1_20250101#maakunta_1_20250101.csv'

In [3]:
# Lähtöväestö joku muu kuin koti maakunta
# Lataa uusimmat avaimet https://stat.fi/fi/luokitukset/corrmaps/kunta_1_20250101%23maakunta_1_20250101
kuntamaakunta_csv = pd.read_csv(KUNTA_MAAKUNTA_CSV, usecols=["sourceCode", "sourceName","targetCode", "targetName"], sep=";")

#Muu kuin oma maakunta
muu_maakunta = kuntamaakunta_csv[(kuntamaakunta_csv['targetCode'].apply(lambda x: OMA_MAAKUNTA not in x))]

muu_maakunta['paate'] = 'KU'
muu_maakunta["kunta"] = muu_maakunta["paate"] + muu_maakunta["sourceCode"]

#Poistetaan erikoismerkit
muu_maakunta["kuntaid"] = muu_maakunta["kunta"].map(lambda x: re.sub(r'\W+', '', x))
#Tehdään lista
MUUT_KUNNAT = muu_maakunta["kuntaid"].tolist()
print(MUUT_KUNNAT)

['KU005', 'KU009', 'KU010', 'KU016', 'KU018', 'KU019', 'KU035', 'KU043', 'KU046', 'KU047', 'KU049', 'KU050', 'KU051', 'KU052', 'KU060', 'KU061', 'KU062', 'KU065', 'KU069', 'KU071', 'KU072', 'KU074', 'KU075', 'KU076', 'KU077', 'KU078', 'KU079', 'KU081', 'KU082', 'KU086', 'KU111', 'KU090', 'KU091', 'KU097', 'KU098', 'KU102', 'KU103', 'KU105', 'KU106', 'KU109', 'KU139', 'KU140', 'KU142', 'KU145', 'KU146', 'KU153', 'KU148', 'KU149', 'KU151', 'KU152', 'KU165', 'KU167', 'KU169', 'KU170', 'KU171', 'KU172', 'KU176', 'KU178', 'KU179', 'KU181', 'KU182', 'KU186', 'KU202', 'KU204', 'KU205', 'KU208', 'KU213', 'KU214', 'KU216', 'KU217', 'KU218', 'KU224', 'KU226', 'KU230', 'KU231', 'KU232', 'KU233', 'KU235', 'KU236', 'KU239', 'KU240', 'KU320', 'KU241', 'KU322', 'KU244', 'KU245', 'KU249', 'KU256', 'KU257', 'KU260', 'KU261', 'KU263', 'KU265', 'KU271', 'KU272', 'KU273', 'KU275', 'KU276', 'KU280', 'KU284', 'KU285', 'KU286', 'KU287', 'KU288', 'KU290', 'KU295', 'KU297', 'KU300', 'KU301', 'KU304', 'KU305', 

In [4]:
query = {
  "query": [
    {
      "code": "Tuloalue",
      "selection": {
        "filter": "item",
        "values": OMA_KUNTA
      }
    },
    {
      "code": "Lähtöalue",
      "selection": {
        "filter": "item",
        "values": MUUT_KUNNAT
      }
    },
    {
      "code": "Ikä",
      "selection": {
        "filter": "item",
        "values": IKÄRYHMÄT
      }
    },
    {
      "code": "Kieli",
      "selection": {
        "filter": "item",
        "values": [
          "SSS"
        ]
      }
    },
    {
      "code": "Vuosi",
      "selection": {
        "filter": "item",
        "values": VUODET
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
resultat = requests.post(POST_URL, headers=headers, json = query)
print(resultat)


<Response [200]>


In [5]:
dataset = pyjstat.Dataset.read(resultat.text)

In [6]:
#Write dataframe
df = dataset.write('dataframe')
#Get all values
df_all = df[(df['Ikä'].str.contains('Yhteensä'))]
#Get men
df_men = df[(df['Ikä'].apply(lambda x: 'Yhteensä' not in x))]
#Get women
df_women = df[(df['Ikä'].apply(lambda x: 'Yhteensä' not in x))]

df.head(200)


,Tuloalue,Lähtöalue,Ikä,Kieli,Vuosi,Tiedot,value
0,Tulo - Kangasala,Lähtö - Alajärvi,Yhteensä,Yhteensä,2020,Kuntien välinen tulomuutto,2
1,Tulo - Kangasala,Lähtö - Alajärvi,Yhteensä,Yhteensä,2021,Kuntien välinen tulomuutto,0
2,Tulo - Kangasala,Lähtö - Alajärvi,Yhteensä,Yhteensä,2022,Kuntien välinen tulomuutto,2
3,Tulo - Kangasala,Lähtö - Alajärvi,Yhteensä,Yhteensä,2023,Kuntien välinen tulomuutto,2
4,Tulo - Kangasala,Lähtö - Alajärvi,Yhteensä,Yhteensä,2024,Kuntien välinen tulomuutto,3
...,...,...,...,...,...,...,...
195,Tulo - Kangasala,Lähtö - Alavus,20 - 24,Yhteensä,2020,Kuntien välinen tulomuutto,0
196,Tulo - Kangasala,Lähtö - Alavus,20 - 24,Yhteensä,2021,Kuntien välinen tulomuutto,0
197,Tulo - Kangasala,Lähtö - Alavus,20 - 24,Yhteensä,2022,Kuntien välinen tulomuutto,0
198,Tulo - Kangasala,Lähtö - Alavus,20 - 24,Yhteensä,2023,Kuntien välinen tulomuutto,0


In [7]:
#Year average
df_group_all = round(df_all.groupby(['Tuloalue']).agg(yht=('value', 'mean')),0)
#Year average men
df_group_men = pd.DataFrame(df_men.groupby(['Tuloalue', 'Ikä']).agg(m=('value', 'mean')))
#Year average women
df_group_women = pd.DataFrame(df_women.groupby(['Tuloalue', 'Ikä']).agg(n=('value', 'mean')))

# Convert groupby object to DataFrame 
df_group_all_1 = df_group_all.reset_index()
df_group_men_1 = df_group_men.reset_index()
df_group_women_1 = df_group_women.reset_index()

df_group_women_1.drop(columns=["Ikä"], inplace=True, errors='ignore')

# Repeat value
df_group_all_1 = pd.DataFrame(np.repeat(df_group_all_1.values, repeats=len(df_group_men_1.index), axis=0), columns=df_group_all_1.columns)

df_group = pd.concat([df_group_men_1,df_group_women_1,df_group_all_1], axis=1)
columns_to_drop = ["Tuloalue"]
df_group.drop(columns=columns_to_drop, inplace=True, errors='ignore')

#Sukupuolien osuudet muutoissa
df_group['m_pro'] = (df_group['m'] / df_group['yht'])
df_group['n_pro'] = (df_group['n'] / df_group['yht'])

#Jaetaan 5, jotta saadaan 1-vuodelle luvut
df_group['m_pro'] = (df_group['m_pro'] / 5)
df_group['n_pro'] = (df_group['n_pro'] / 5)

print(df_group)


        Ikä         m         n  yht     m_pro     n_pro
0     0 - 4  0.087719  0.087719  2.0  0.008772  0.008772
1   10 - 14  0.036491  0.036491  2.0  0.003649  0.003649
2   15 - 19  0.146667  0.146667  2.0  0.014667  0.014667
3   20 - 24  0.383860  0.383860  2.0  0.038386  0.038386
4   25 - 29  0.251930  0.251930  2.0  0.025193  0.025193
5   30 - 34  0.181754  0.181754  2.0  0.018175  0.018175
6   35 - 39  0.123509  0.123509  2.0  0.012351  0.012351
7   40 - 44  0.088421  0.088421  2.0  0.008842  0.008842
8   45 - 49  0.068070  0.068070  2.0  0.006807  0.006807
9     5 - 9  0.066667  0.066667  2.0  0.006667  0.006667
10  50 - 54  0.063158  0.063158  2.0  0.006316  0.006316
11  55 - 59  0.059649  0.059649  2.0  0.005965  0.005965
12  60 - 64  0.072281  0.072281  2.0  0.007228  0.007228
13  65 - 69  0.061754  0.061754  2.0  0.006175  0.006175
14  70 - 74  0.044912  0.044912  2.0  0.004491  0.004491
15     75 -  0.054035  0.054035  2.0  0.005404  0.005404


In [8]:
#Generate years
ikä = [x for x in range(100)]
df_ikä = pd.DataFrame(ikä, columns=['Ikä']).apply(pd.to_numeric)

conditions = [
    (df_ikä['Ikä'] <= 4),
    (df_ikä['Ikä'] > 4) & (df_ikä['Ikä'] <= 9),
    (df_ikä['Ikä'] > 9) & (df_ikä['Ikä'] <= 14),
    (df_ikä['Ikä'] > 14) & (df_ikä['Ikä'] <= 19),
    (df_ikä['Ikä'] > 19) & (df_ikä['Ikä'] <= 24),
    (df_ikä['Ikä'] > 24) & (df_ikä['Ikä'] <= 29),
    (df_ikä['Ikä'] > 29) & (df_ikä['Ikä'] <= 34),
    (df_ikä['Ikä'] > 34) & (df_ikä['Ikä'] <= 39),
    (df_ikä['Ikä'] > 39) & (df_ikä['Ikä'] <= 44),
    (df_ikä['Ikä'] > 44) & (df_ikä['Ikä'] <= 49),
    (df_ikä['Ikä'] > 49) & (df_ikä['Ikä'] <= 54),
    (df_ikä['Ikä'] > 54) & (df_ikä['Ikä'] <= 59),
    (df_ikä['Ikä'] > 59) & (df_ikä['Ikä'] <= 64),
    (df_ikä['Ikä'] > 64) & (df_ikä['Ikä'] <= 69),
    (df_ikä['Ikä'] > 69) & (df_ikä['Ikä'] <= 74),
    (df_ikä['Ikä'] > 74)
]

# create a list of the values we want to assign for each condition
values = ['0 - 4', '5 - 9', '10 - 14', '15 - 19', '20 - 24', '25 - 29', 
          '30 - 34', '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64', '65 - 69', '70 - 74', '75 -']

df_ikä['Ikä_lk'] = np.select(conditions, values, default='Other')

# Merging on multiple columns
res = pd.merge(df_ikä, df_group, left_on=['Ikä_lk'], right_on=['Ikä'], how='inner')

# *Generoidaan sarakenimet;
men = pd.DataFrame(['mtt' + str(x) for x in range(100)])
women = pd.DataFrame(['ntt' + str(x) for x in range(100)])

res['men'] = men
res['women'] = women



print(res)



    Ikä_x Ikä_lk  Ikä_y         m         n  yht     m_pro     n_pro    men  \
0       0  0 - 4  0 - 4  0.087719  0.087719  2.0  0.008772  0.008772   mtt0   
1       1  0 - 4  0 - 4  0.087719  0.087719  2.0  0.008772  0.008772   mtt1   
2       2  0 - 4  0 - 4  0.087719  0.087719  2.0  0.008772  0.008772   mtt2   
3       3  0 - 4  0 - 4  0.087719  0.087719  2.0  0.008772  0.008772   mtt3   
4       4  0 - 4  0 - 4  0.087719  0.087719  2.0  0.008772  0.008772   mtt4   
..    ...    ...    ...       ...       ...  ...       ...       ...    ...   
95     95   75 -   75 -  0.054035  0.054035  2.0  0.005404  0.005404  mtt95   
96     96   75 -   75 -  0.054035  0.054035  2.0  0.005404  0.005404  mtt96   
97     97   75 -   75 -  0.054035  0.054035  2.0  0.005404  0.005404  mtt97   
98     98   75 -   75 -  0.054035  0.054035  2.0  0.005404  0.005404  mtt98   
99     99   75 -   75 -  0.054035  0.054035  2.0  0.005404  0.005404  mtt99   

    women  
0    ntt0  
1    ntt1  
2    ntt2  
3  

In [9]:
#Create final Excelformat
df_out_men = res[["m_pro", "men"]].copy()
df_out_men = df_out_men.set_index('men').transpose()

df_out_women = res[["n_pro", "women"]].copy()
df_out_women = df_out_women.set_index('women').transpose()

df_out_all = res[["yht"]].copy()
df_out_all_first = df_out_all.iloc[:1]
df_out_all_first.rename(columns={'yht': 'tmtyht'}, inplace=True)

list_of_years = list(zip(ENNUSTEVUODET))
list_of_years

df_vuodet = pd.DataFrame(list_of_years,
                  columns=['vuosi'])

df_out_tyyppi = pd.DataFrame(np.array([['1']]))
df_out_tyyppi.columns = ['tyyppi']
df_vuodet.columns = ['vuosi']
df_out_alue = pd.DataFrame(np.array([['1']]))
df_out_alue.columns = ['alue']



final = pd.concat([
    df_out_alue.reset_index(drop=True),
    df_out_tyyppi.reset_index(drop=True),
    df_out_all_first.reset_index(drop=True),
    df_out_men.reset_index(drop=True),
    df_out_women.reset_index(drop=True)], axis=1)

# Repeat value
final2 = pd.DataFrame(np.repeat(final.values, repeats=len(df_vuodet), axis=0), columns=final.columns)

final3 = pd.concat([df_vuodet, final2], axis=1)

print(final3)

final3.to_excel(file_exc, sheet_name='Sheet_name_1', index=False)



   vuosi alue tyyppi tmtyht      mtt0      mtt1      mtt2      mtt3      mtt4  \
0   2025    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
1   2026    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
2   2027    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
3   2028    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
4   2029    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
5   2030    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
6   2031    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
7   2032    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
8   2033    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
9   2034    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   
10  2035    1      1    2.0  0.008772  0.008772  0.008772  0.008772  0.008772   

        mtt5  ...     ntt90